# Geração da Tabela Base de Origem

In [1]:
# Esse processo preciso rodar diariamente e localmente

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import streamlit as st

def get_fii_table():
    url = "https://www.fundsexplorer.com.br/ranking"

    options = Options()
    # options.add_argument("--headless=new")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(url)
        time.sleep(2)  # espera inicial

        # --- 0) Tenta fechar/aceitar o cookie banner por vários caminhos ---
        cookie_selectors = [
            'button[data-test="accept"]',
            'button#hs-eu-confirmation-button',             # exemplos comuns
            'button[aria-label*="aceitar"]',
            'button:contains("Aceitar")',                   # fallback textual (pode não funcionar no CSS)
            'div#hs-en-cookie-confirmation-buttons-area button',
            'button:contains("Aceitar todos")'
        ]
        # tentar por texto também (mais confiável)
        texts_to_try = ["Aceitar todos", "Aceitar", "OK", "Fechar"]

        # 1) tentar clique por seletores diretos
        for sel in cookie_selectors:
            try:
                els = driver.find_elements(By.CSS_SELECTOR, sel)
                if els:
                    for e in els:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", e)
                            driver.execute_script("arguments[0].click();", e)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    # se algum botão foi clicado, pausa e tenta prosseguir
                    time.sleep(0.6)
            except Exception:
                pass

        # 2) tentar clicar por texto (procura por botões/links)
        for txt in texts_to_try:
            try:
                candidates = driver.find_elements(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}') or //a[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}')]]")
                if candidates:
                    for c in candidates:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", c)
                            driver.execute_script("arguments[0].click();", c)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    time.sleep(0.6)
            except Exception:
                pass

        # 3) se ainda existir o elemento de cookie conhecido, remove via JS (forçado)
        try:
            driver.execute_script("""
                var el = document.getElementById('hs-en-cookie-confirmation-buttons-area');
                if (el) { el.remove(); }
                var el2 = document.querySelector('[id^=hs-en-cookie]'); if(el2) el2.remove();
                var overlays = document.querySelectorAll('.cookie, .cookies, .hs-cookie-banner'); 
                overlays.forEach(e => e.remove());
            """)
            time.sleep(0.4)
        except Exception:
            pass

        # --- 4) abrir o menu de colunas (scroll + click via JS para evitar intercept) ---
        botao_colunas = wait.until(EC.presence_of_element_located((By.ID, "colunas-ranking__select-button")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_colunas)
        try:
            # preferencial: click via JS para evitar intercept
            driver.execute_script("arguments[0].click();", botao_colunas)
        except Exception:
            # fallback: webdriver click
            botao_colunas.click()
        time.sleep(0.6)

        # --- 5) clicar em "todos" (label) ---
        label_todos = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'label[for="colunas-ranking__todos"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", label_todos)
        try:
            driver.execute_script("arguments[0].click();", label_todos)
        except Exception:
            label_todos.click()
        time.sleep(1.0)  # deixa o JS atualizar a tabela

        # --- 6) esperar a tabela estar populada ---
        def tabela_populada(driver):
            try:
                rows = driver.find_elements(By.CSS_SELECTOR, ".default-fiis-table__container__table tbody tr")
                # contar linhas não-vazias
                count = 0
                for r in rows:
                    tds = r.find_elements(By.TAG_NAME, "td")
                    if any(td.text.strip() for td in tds):
                        count += 1
                return count > 5  # ajuste se precisar
            except:
                return False

        wait.until(tabela_populada)

        # --- 7) pegar HTML da tabela ---
        tabela = driver.find_element(By.CSS_SELECTOR, ".default-fiis-table__container__table")
        html = tabela.get_attribute("outerHTML")
        df = pd.read_html(html)[0]
        return df

    finally:
        driver.quit()

#Call the function to test
df_fiis = get_fii_table()

def carregar_dados(df=df_fiis):

    df = df.dropna(subset=[
        'P/VP',
        'DY (3M) Acumulado',
        'DY (6M) Acumulado',
        'DY (12M) Acumulado',
        'Liquidez Diária (R$)',
        'Patrimônio Líquido',
        'Num. Cotistas',
        'Preço Atual (R$)'
    ])

    df['P/VP'] = df['P/VP'] / 100

    for col in ['DY (3M) Acumulado', 'DY (6M) Acumulado', 'DY (12M) Acumulado']:
        df[col] = (
            df[col].astype(str)
            .str.replace('%', '', regex=False)
            .str.replace('.', '', regex=False)
            .str.replace(',', '.', regex=False)
            .astype(float)
        )

    df['Liquidez Diária (R$)'] = (
        df['Liquidez Diária (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Patrimônio Líquido'] = (
        df['Patrimônio Líquido']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Num. Cotistas'] = (
        df['Num. Cotistas']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000
    )

    df['Preço Atual (R$)'] = (
        df['Preço Atual (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df['Último Dividendo'] = (
        df['Último Dividendo']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df.rename(columns={
        'Liquidez Diária (R$)': 'Liquidez Diária (milhões R$)',
        'Patrimônio Líquido': 'Patrimônio Líquido (milhões R$)',
        'Num. Cotistas': 'Num. Cotistas (milhares)'
    }, inplace=True)

    return df

df_fiis = carregar_dados(df_fiis)
today_str = pd.Timestamp.today().strftime('%Y-%m-%d')

ajustes_manuais_segmento = pd.read_csv('ajustes_manuais_segmento.csv')

df_fiis = df_fiis.merge(ajustes_manuais_segmento, on='Fundos', how='left', suffixes=('', '_novo'))
df_fiis['Setor'] = df_fiis['Setor_novo'].combine_first(df_fiis['Setor'])
df_fiis.drop(columns=['Setor_novo'], inplace=True)

# Salvando os Top 10 FIIs Descontados com Qualidade
def filtrar_fiis_descontados_com_qualidade(df):
    filtros = (
        (df["P/VP"] >= 0.85) &
        (df["P/VP"] < 1.0) &
        (df["DY (3M) Acumulado"] >= 3) &
        (df["DY (6M) Acumulado"] >= 6) &
        (df["DY (12M) Acumulado"] >= 12) &
        (df["Liquidez Diária (milhões R$)"] >= 1) &
        (df["Patrimônio Líquido (milhões R$)"] >= 500) &
        (df["Num. Cotistas (milhares)"] >= 10)
    )
    return df[filtros].copy()

df_filtrados = filtrar_fiis_descontados_com_qualidade(df_fiis)

df_top10 = (
    df_filtrados
    .sort_values("DY (12M) Acumulado", ascending=False)
    .head(10)
) 

C:\Users\danie\AppData\Local\Temp\ipykernel_16980\522615446.py:128: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_16980\522615446.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['P/VP'] = df['P/VP'] / 100
C:\Users\danie\AppData\Local\Temp\ipykernel_16980\522615446.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [2]:
import yfinance as yf
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from glob import glob

dfs = glob('df_fiis/df_fiis*')
lim = 0

# ==========================
# PARÂMETROS
# ==========================
PVP_MIN = 0.85
PVP_MAX = 1.00
LIQUIDEZ_MIN = 1.0      # R$ mi/dia
JANELA_QUEDA = 10
CDI = 0.15
SELIC_ANUAL = (1 + CDI) * (1 - 0.225)
PATRIMONIO_MIN = 500  # R$ 500 mi
NUM_COTISTAS_MIN = 10_000
DY_3M_MIN = 3
DY_6M_MIN = 6
DY_12M_MIN = 12

coluna_score = []
coluna_bloqueios = []
coluna_motivos = []
fiis_base = []

colunas_utilizadas = [
    'Fundos', 'Setor', 'Preço Atual (R$)', 'Liquidez Diária (milhões R$)',
    'P/VP', 'Último Dividendo', 'Dividend Yield', 'DY (3M) Acumulado',
    'DY (6M) Acumulado', 'DY (12M) Acumulado', 'DY Ano',
    'Patrimônio Líquido (milhões R$)', 'Quant. Ativos',
    'Num. Cotistas (milhares)'
]

df_fiis = df_fiis[colunas_utilizadas]

for i in df_fiis.Fundos.unique():
    try:
        ticker = yf.Ticker(f"{i}.SA")
        hist = ticker.history(period="1y")

        if len(hist) < 200:
            raise ValueError("Histórico insuficiente")

        retornos = hist["Close"].pct_change()

        # =====================================================
        # 🔥 CRITÉRIO DE VOLATILIDADE / EVENTO ATÍPICO (AJUSTADO)
        # =====================================================
        retorno_10d = (1 + retornos).rolling(JANELA_QUEDA).apply(
            lambda x: x.prod() - 1,
            raw=False
        )

        if pd.isna(retorno_10d.iloc[-1]):
            raise ValueError("Histórico insuficiente para janela de 10 dias")

        retorno_10d_atual = retorno_10d.iloc[-1]
        percentil_queda = (retorno_10d <= retorno_10d_atual).mean()

        # =====================================================
        score = 0
        bloqueios = []
        motivos = []

        # P/VP
        if PVP_MIN <= df_fiis[df_fiis["Fundos"] == i]["P/VP"].iloc[0] <= PVP_MAX:
            score += 1
            motivos.append("✅ Preço dentro da faixa saudável")
        else:
            bloqueios.append("❌ Preço fora da faixa saudável")

        # Liquidez
        if df_fiis[df_fiis["Fundos"] == i]["Liquidez Diária (milhões R$)"].iloc[0] >= LIQUIDEZ_MIN:
            score += 1
            motivos.append("✅ Liquidez adequada para entrada e saída")
        else:
            bloqueios.append("❌ Liquidez inadequada para operação segura")

        # Evento atípico de curto prazo
        if percentil_queda >= 0.15:
            score += 1
            motivos.append("✅ Movimentação recente compatível com o histórico do ativo")
        else:
            bloqueios.append("❌ Movimentação recente atípica vs histórico do ativo")

        # Patrimônio
        if df_fiis[df_fiis["Fundos"] == i]["Patrimônio Líquido (milhões R$)"].iloc[0] >= PATRIMONIO_MIN:
            score += 1
            motivos.append("✅ Escala patrimonial adequada")
        else:
            bloqueios.append("❌ Escala patrimonial abaixo do recomendado")

        # Cotistas
        num_cotistas = df_fiis[df_fiis["Fundos"] == i]["Num. Cotistas (milhares)"].iloc[0] * 1000
        if num_cotistas >= NUM_COTISTAS_MIN:
            score += 1
            if num_cotistas >= 150_000:
                motivos.append("✅ Base de cotistas consolidada")
            else:
                motivos.append("✅ Base de cotistas adequada")
        else:
            bloqueios.append("❌ Base de cotistas pouco representativa")

        # DY
        if (
            df_fiis[df_fiis["Fundos"] == i]["DY (3M) Acumulado"].iloc[0] >= DY_3M_MIN
            and df_fiis[df_fiis["Fundos"] == i]["DY (6M) Acumulado"].iloc[0] >= DY_6M_MIN
            and df_fiis[df_fiis["Fundos"] == i]["DY (12M) Acumulado"].iloc[0] >= DY_12M_MIN
        ):
            score += 1
            motivos.append("✅ Distribuição de rendimentos consistente")
        else:
            bloqueios.append("❌ Distribuição de rendimentos inconsistente")

        coluna_score.append(score)
        coluna_bloqueios.append(bloqueios)
        coluna_motivos.append(motivos)
        fiis_base.append(i)

    except Exception as e:
        coluna_score.append(0)
        coluna_bloqueios.append(["Informação insuficiente"])
        coluna_motivos.append([str(e)])
        fiis_base.append(i)

df_fiis = df_fiis[colunas_utilizadas]
df_fiis = df_fiis.drop_duplicates()
df_fiis['Motivos'] = coluna_motivos
df_fiis['Bloqueios'] = coluna_bloqueios
df_fiis['Score'] = coluna_score

df_fiis.to_parquet(f'df_fiis/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'df_fiis/df_fiis_{today_str}.parquet', index=False)


# df_top10.to_parquet(f'hist_top10/hist_top10_{today_str}.parquet', index=False)
# df_top10.to_parquet(f'hist_top10/df_top10.parquet', index=False)
# df_top10.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_top10.parquet', index=False)


$ARCT11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$ASMT11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BBFI11B.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BICR11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BLMC11.SA: possibly delisted; no price data found  (period=1y)
$BLMR11.SA: possibly delisted; no price data found  (period=1y)
$BPRP11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BREV11.SA: possibly delisted; no price data found  (period=1y)
$BRIX11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BRLA11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data

In [3]:
df_fiis.to_parquet(f'df_fiis/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'df_fiis/df_fiis_{today_str}.parquet', index=False)

# Construindo a base completa

In [4]:
# x = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
# x['data'] = '2025-12-21'
# x.to_parquet('df_fiis/base_completa_df_fiis.parquet')

In [5]:
remover = []

In [6]:
from glob import glob

dfs = glob('df_fiis/df_fiis_*')
df_inicial = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
df_inicial = df_inicial[~df_inicial.data.isin(remover)]
datas_ja_inseridas = df_inicial.data.unique()

for i in dfs:
    data = i.split('.')[0].split('_')[-1]
    if data not in datas_ja_inseridas:
        print('Inserindo :',data)
        df_data = pd.read_parquet(i)
        df_data['data'] = data
        df_inicial = pd.concat([df_inicial,df_data])   
    else:
        print(data,'já inserida')

df_inicial.to_parquet('df_fiis/base_completa_df_fiis.parquet')
df_inicial.data.unique()


2025-12-21 já inserida
2025-12-22 já inserida
2025-12-29 já inserida
2025-12-30 já inserida
2026-01-03 já inserida
2026-01-04 já inserida
2026-01-05 já inserida
2026-01-06 já inserida
2026-01-07 já inserida
2026-01-08 já inserida
Inserindo : 2026-01-09


array(['2025-12-21', '2025-12-22', '2025-12-29', '2025-12-30',
       '2026-01-03', '2026-01-04', '2026-01-05', '2026-01-06',
       '2026-01-07', '2026-01-08', '2026-01-09'], dtype=object)

In [7]:
x = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
x[x.Fundos == 'LIFE11']

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (milhões R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY Ano,Patrimônio Líquido (milhões R$),Quant. Ativos,Num. Cotistas (milhares),Motivos,Bloqueios,Score,data
235,LIFE11,Imóveis Comerciais - Outros,7.96,0.763797,0.80,0.12,"1,48 %",4.43,8.76,18.27,"18,27 %",393.113429,14,18.2,[✅ Queda recente compatível com o histórico do...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2025-12-21
235,LIFE11,Imóveis Comerciais - Outros,7.96,0.763797,0.80,0.12,"1,48 %",4.43,8.76,18.27,"18,27 %",393.113429,14,18.2,[✅ Queda recente compatível com o histórico do...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2025-12-22
235,LIFE11,Imóveis Comerciais - Outros,8.33,0.838834,0.83,0.12,"1,48 %",4.43,8.76,18.27,"18,27 %",393.113429,14,18.2,[✅ Queda recente compatível com o histórico do...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2025-12-29
235,LIFE11,Imóveis Comerciais - Outros,8.44,0.838834,0.83,0.12,"1,48 %",4.43,8.76,18.27,"18,27 %",393.113429,14,18.2,[✅ Queda recente compatível com o histórico do...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2025-12-30
236,LIFE11,Imóveis Comerciais - Outros,8.30,0.798632,0.86,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,"[✅ Preço dentro da faixa saudável, ✅ Queda rec...","[❌ Liquidez inadequada para operação segura, ❌...",4,2026-01-03
236,LIFE11,Imóveis Comerciais - Outros,8.30,0.821573,0.86,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,"[✅ Preço dentro da faixa saudável, ✅ Movimenta...","[❌ Liquidez inadequada para operação segura, ❌...",4,2026-01-04
236,LIFE11,Imóveis Comerciais - Outros,8.35,0.844253,0.84,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2026-01-05
236,LIFE11,Imóveis Comerciais - Outros,8.39,0.820866,0.84,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2026-01-06
236,LIFE11,Imóveis Comerciais - Outros,8.23,0.793202,0.84,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2026-01-07
236,LIFE11,Imóveis Comerciais - Outros,8.25,0.813373,0.83,0.12,"1,41 %",4.37,8.73,17.78,"1,41 %",393.113429,14,18.2,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",3,2026-01-08


# Pegando os melhores e pegando os que não passam no critério

In [8]:
#lendo o arquivo mais atual 
from glob import glob
# dfs = glob('df_fiis/df_fiis_*')

# ultimo_arquivo = dfs[-1]
ultimo_arquivo = 'df_fiis/df_fiis.parquet'

base_atual = pd.read_parquet(ultimo_arquivo)

score_perfeitos = base_atual[base_atual.Score == 6]
score_bons = base_atual[(base_atual.Score >= 4) & (base_atual.Score < 6)]
score_obs = base_atual[(base_atual.Score == 3)]
score_ruins = base_atual[(base_atual.Score.isin([1,2]))]
score_horriveis = base_atual[(base_atual.Score == 0) & (base_atual.Motivos.apply(lambda x : 'Histórico insuficiente' not in x))]

print('Perfeitos :',score_perfeitos.shape)
print('Bons :',score_bons.shape)
print('Obs :',score_obs.shape)
print('Ruins :',score_ruins.shape)
print('Horriveis :',score_horriveis.shape)


Perfeitos : (20, 17)
Bons : (80, 17)
Obs : (68, 17)
Ruins : (193, 17)
Horriveis : (8, 17)


In [9]:
score_perfeitos.Fundos.unique()

array(['BTHF11', 'CPSH11', 'CPTS11', 'GGRC11', 'HGCR11', 'JSAF11',
       'KNCA11', 'KNHF11', 'KNSC11', 'MCCI11', 'MCRE11', 'RBRX11',
       'RBRY11', 'RECR11', 'RZAK11', 'RZTR11', 'VCJR11', 'VGIR11',
       'VRTA11', 'XPCI11'], dtype=object)

In [10]:
score_perfeitos.sort_values(['DY (3M) Acumulado'],ascending=False).sort_values(['DY (6M) Acumulado'],ascending=False).sort_values('P/VP').sort_values(['DY (12M) Acumulado'],ascending=False).head()

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (milhões R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY Ano,Patrimônio Líquido (milhões R$),Quant. Ativos,Num. Cotistas (milhares),Motivos,Bloqueios,Score
356,RZAK11,Papéis,84.56,1.650465,0.97,1.10,"1,34 %",4.05,8.45,19.03,"19,03 %",769.306840,14,45.037,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
415,VGIR11,Papéis,9.80,4.792840,1.00,0.13,"1,36 %",4.14,8.43,16.61,"16,61 %",1430.805115,1,255.796,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
254,MCRE11,Papéis,9.51,4.407367,0.94,0.11,"1,22 %",3.76,7.75,16.35,"16,35 %",1119.945715,1,86.943,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
329,RBRY11,Papéis,98.40,5.803169,0.99,1.25,"1,29 %",3.94,7.93,15.72,"15,72 %",1271.058738,14,67.221,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
407,VCJR11,Papéis,79.14,1.766330,0.85,0.77,"0,95 %",3.27,6.72,15.42,"0,95 %",1387.626192,14,27.375,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6


In [11]:
# FII armadilhas
score_horriveis.sort_values('Dividend Yield',ascending=False).head()

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (milhões R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY Ano,Patrimônio Líquido (milhões R$),Quant. Ativos,Num. Cotistas (milhares),Motivos,Bloqueios,Score
333,RCFF11,Fundo de Fundos,80.00,0.016000,0.81,3.55,"3,76 %",0.00,3.76,9.17,"9,17 %",31.863620,14,0.129,[],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",0
374,SJAU11,Imóveis Industriais e Logísticos,0.61,0.018108,0.42,20.13,"231,65 %",0.00,0.00,848.21,"848,21 %",11.309911,14,0.483,[],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",0
120,FIIP11,Imóveis Industriais e Logísticos,149.99,0.327925,0.75,1.40,"0,88 %",2.61,5.39,11.35,"0,88 %",185.473049,7,6.043,[],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",0
97,DAYM11,Lajes Corporativas,3.86,0.017809,0.39,0.04,"0,80 %",2.53,3.31,3.31,"3,31 %",170.323099,6,0.777,[],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",0
27,BGRB11,Lajes Corporativas,101.20,0.020318,0.84,0.81,"0,68 %",0.68,0.82,3.05,"0,68 %",232.005448,1,0.373,[],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",0


# Alerta Movimentação P/VP

In [12]:
base_completa = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
# Preciso comparar um dia com o outro para ver os pvp
dfs = glob('df_fiis/df_fiis_*')
dia_anterior = dfs[-2].split('.')[0].split('_')[-1]
dia_hoje =  dfs[-1].split('.')[0].split('_')[-1]
print(dia_anterior)
print(dia_hoje)
# pegar o pvp dos fiis nesses dias 

print('Desceram para o critério')
for i in base_completa.Fundos.unique():
    try:
        pvp_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
        pvp_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
        if pvp_atual < 1 and pvp_anterior >= 1:
            print(i, 'o pvp era', pvp_anterior,'e agora é ',pvp_atual)
    except:
        print(i,'deu erro') 

print('________________________')
print('Subiram para o critério')
for i in base_completa.Fundos.unique():
    try:
        pvp_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
        pvp_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
        if pvp_atual >= 0.85 and pvp_anterior < 0.85:
            print(i, 'o pvp era', pvp_anterior,'e agora é ',pvp_atual)
    except:
        print(i,'deu erro') 




2026-01-08
2026-01-09
Desceram para o critério
BGRB11 o pvp era 1.0 e agora é  0.84
MALL11 deu erro
PQAG11 o pvp era 1.02 e agora é  0.98
RECD11 o pvp era 1.02 e agora é  0.94
SNFZ11 o pvp era 1.01 e agora é  0.99
________________________
Subiram para o critério
CXCI11 o pvp era 0.84 e agora é  0.85
LASC11 o pvp era 0.84 e agora é  0.85
MALL11 deu erro
MGHT11 o pvp era 0.83 e agora é  0.87
PMLL11 o pvp era 0.84 e agora é  0.85
VCJR11 o pvp era 0.84 e agora é  0.85


# Entrando no Score perfeito

In [13]:
import pandas as pd
from glob import glob

base_completa = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')

# Datas
dfs = glob('df_fiis/df_fiis_*')
dfs = sorted(dfs)

dia_anterior = dfs[-2].split('.')[0].split('_')[-1]
dia_hoje = dfs[-1].split('.')[0].split('_')[-1]

print(dia_anterior)
print(dia_hoje)

mensagem = ""

SEPARADOR = "\n══════════════════\n\n"

# ======================================================
# 🟡 OBSERVAÇÃO → 🟢 APROVADO
# ======================================================
for i in base_completa.Fundos.unique():
    try:
        score_anterior = base_completa[
            (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        score_atual = base_completa[
            (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        if score_atual == 6 and score_anterior != 6:
            #print(i, 'foi de 🟡 OBSERVAÇÃO → 🟢 APROVADO')

            mensagem += (
                f"{i}\n"
                f"🔼 Mudança : 🟡 Observação → 🟢 Aprovado\n"
                f"Motivos que passaram a atender aos critérios:\n"
            )

            motivos_hoje = base_completa[
                (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
            ]['Motivos'].iloc[0]

            motivos_ontem = base_completa[
                (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
            ]['Motivos'].iloc[0]

            for motivo in motivos_hoje:
                if motivo not in motivos_ontem:
                    # print(motivo)
                    mensagem += f"- {motivo}\n"

            mensagem += SEPARADOR

    except Exception as e:
        print(i, 'deu erro:', e)

# ======================================================
# 🟢 APROVADO → 🟡 OBSERVAÇÃO
# ======================================================
for i in base_completa.Fundos.unique():
    try:
        score_anterior = base_completa[
            (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        score_atual = base_completa[
            (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        if score_atual < 6 and score_atual >= 3 and score_anterior == 6:
            #print(i, 'foi de 🟢 APROVADO → 🟡 OBSERVAÇÃO')

            mensagem += (
                f"{i}\n"
                f"⚠️ Mudança : 🟢 Aprovado → 🟡 Observação\n"
                f"Critérios que deixaram de ser atendidos:\n"
            )

            bloqueios_hoje = base_completa[
                (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
            ]['Bloqueios'].iloc[0]

            bloqueios_ontem = base_completa[
                (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
            ]['Bloqueios'].iloc[0]

            for motivo in bloqueios_hoje:
                if motivo not in bloqueios_ontem:
                    #print(motivo)
                    mensagem += f"- {motivo}\n"

            mensagem += SEPARADOR

    except Exception as e:
        print(i, 'deu erro:', e)

# ======================================================
# 🟡 OBSERVAÇÃO → 🔴 BLOQUEADO
# ======================================================
for i in base_completa.Fundos.unique():
    try:
        score_anterior = base_completa[
            (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        score_atual = base_completa[
            (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
        ]['Score'].iloc[0]

        if score_atual < 3 and score_anterior >= 3 and score_anterior != 6:
            #print(i, 'foi de 🟡 OBSERVAÇÃO → 🔴 BLOQUEADO')

            mensagem += (
                f"{i}\n"
                f"⛔ Mudança : 🟡 Observação → 🔴 Bloqueado\n"
                f"Bloqueios identificados no período:\n"
            )

            bloqueios_hoje = base_completa[
                (base_completa.data == dia_hoje) & (base_completa.Fundos == i)
            ]['Bloqueios'].iloc[0]

            bloqueios_ontem = base_completa[
                (base_completa.data == dia_anterior) & (base_completa.Fundos == i)
            ]['Bloqueios'].iloc[0]

            for motivo in bloqueios_hoje:
                if motivo not in bloqueios_ontem:
                    # print(motivo)
                    mensagem += f"- {motivo}\n"

            mensagem += SEPARADOR

    except Exception as e:
        print(i, 'deu erro:', e)

print("\nMENSAGEM FINAL:\n")
print(mensagem)


2026-01-08
2026-01-09
MALL11 deu erro: single positional indexer is out-of-bounds
MALL11 deu erro: single positional indexer is out-of-bounds
MALL11 deu erro: single positional indexer is out-of-bounds

MENSAGEM FINAL:

VCJR11
🔼 Mudança : 🟡 Observação → 🟢 Aprovado
Motivos que passaram a atender aos critérios:
- ✅ Preço dentro da faixa saudável

══════════════════

BTCI11
⚠️ Mudança : 🟢 Aprovado → 🟡 Observação
Critérios que deixaram de ser atendidos:
- ❌ Movimentação recente atípica vs histórico do ativo

══════════════════

ARXD11
⛔ Mudança : 🟡 Observação → 🔴 Bloqueado
Bloqueios identificados no período:
- ❌ Preço fora da faixa saudável
- ❌ Movimentação recente atípica vs histórico do ativo

══════════════════

DAMA11
⛔ Mudança : 🟡 Observação → 🔴 Bloqueado
Bloqueios identificados no período:
- ❌ Movimentação recente atípica vs histórico do ativo

══════════════════

FLCR11
⛔ Mudança : 🟡 Observação → 🔴 Bloqueado
Bloqueios identificados no período:
- ❌ Preço fora da faixa saudável
- ❌ Mo

In [27]:
def novo_preco_medio(qtd_atual, preco_medio_atual, qtd_compra, preco_compra):
    total_investido = (qtd_atual * preco_medio_atual) + (qtd_compra * preco_compra)
    qtd_total = qtd_atual + qtd_compra
    return total_investido / qtd_total

# Exemplo
pm = novo_preco_medio(
    qtd_atual=270,
    preco_medio_atual=9.49,
    qtd_compra=150,
    preco_compra=7
)

print(f"Novo preço médio: R$ {pm:.2f}")


Novo preço médio: R$ 8.60


# Pegando Historicos dos preços

In [14]:
import yfinance as yf

ticker = yf.Ticker("CACR11.SA")
hist = ticker.history(period="12mo")  # ou "3mo", "6mo", "1y"
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-09 00:00:00-03:00,72.130543,72.882166,72.130543,72.628807,9598,0.00,0.0
2025-01-10 00:00:00-03:00,72.628786,73.464860,71.497132,73.380409,8629,0.00,0.0
2025-01-13 00:00:00-03:00,73.304410,73.304410,72.628791,72.755470,10333,0.00,0.0
2025-01-14 00:00:00-03:00,72.755461,73.042595,72.595000,72.679451,8810,0.00,0.0
2025-01-15 00:00:00-03:00,72.637245,74.537417,72.637245,73.676003,8992,0.00,0.0
...,...,...,...,...,...,...,...
2026-01-02 00:00:00-03:00,79.900002,79.900002,77.000000,78.150002,19730,1.35,0.0
2026-01-05 00:00:00-03:00,78.059998,78.940002,77.160004,78.019997,17524,0.00,0.0
2026-01-06 00:00:00-03:00,77.510002,78.830002,77.510002,77.820000,15244,0.00,0.0


# Envio por whatsapp

In [65]:
import time
import pyautogui
import pyperclip
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ======================================================
# CONFIGURAÇÃO DO CHROME (mantém login do WhatsApp)
# ======================================================
options = Options()
options.add_argument(r"--user-data-dir=C:\selenium\chrome_profile_whatsapp")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(options=options)

# ======================================================
# LISTA DE USUÁRIOS
# ======================================================
lista_usuarios = {
    "5513981832920": ["Daniel"],
    "5527998976226": ["Esther"]
}

# ======================================================
# ESPERA PARA ABRIR O WHATSAPP WEB (LOGIN)
# ======================================================
time.sleep(3)

# ======================================================
# ENVIO DAS MENSAGENS
# ======================================================
for numero, dados in lista_usuarios.items():

    # Copia a mensagem com emojis para o clipboard
    mensagem_nome = f'Olá {dados[0]}!\n'+mensagem
    pyperclip.copy(mensagem_nome)

    # Abre o chat do WhatsApp
    link = f"https://wa.me/{numero}"
    driver.get(link)


    wait = WebDriverWait(driver, 15)

    # Botão "Continuar para o chat"
    btn_continuar = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[2]/div[4]/a[2]/span")
        )
    )
    btn_continuar.click()

    # Aguarda o chat carregar
    time.sleep(8)

    # Cola a mensagem (CTRL + V) e envia
    pyautogui.hotkey("ctrl", "v")
    time.sleep(1)
    pyautogui.press("enter")

    # Fecha a aba
    time.sleep(1)
    pyautogui.hotkey("ctrl", "w")
    time.sleep(2)
    pyautogui.press('enter')

# ======================================================

# FINALIZA
# ======================================================
driver.quit()


# Acompanhamento da minha carteira

In [29]:
df_fiis[(df_fiis['DY (6M) Acumulado'] > 6)&
        (df_fiis['DY (3M) Acumulado'] > 3)].sort_values('DY (12M) Acumulado',ascending=False).head(5)

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (milhões R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY Ano,Patrimônio Líquido (milhões R$),Quant. Ativos,Num. Cotistas (milhares),Motivos,Bloqueios,Score
443,WSEC11,Multiestratégia,1.41,0.001533,0.96,1.46,"0,00 %",319.54,349.81,715.34,"752,24 %",3.603091,14,0.918,"[✅ Preço dentro da faixa saudável, ✅ Movimenta...","[❌ Liquidez inadequada para operação segura, ❌...",3
47,BRIM11,Fundo de Desenvolvimento,267.67,0.013383,1.31,92.64,"29,12 %",41.51,76.07,178.43,"29,12 %",21.385930,2,0.721,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",2
126,FMOF11,Lajes Corporativas,36.11,0.004630,0.80,1.40,"3,69 %",106.74,109.59,116.40,"3,69 %",25.321999,2,0.363,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",2
324,RBRI11,Fundo de Desenvolvimento,491.18,0.002204,1.01,103.82,"0,00 %",13.80,43.01,83.95,"0,00 %",68.060009,1,0.130,[✅ Movimentação recente compatível com o histó...,"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",2
283,OULG11,Imóveis Industriais e Logísticos,24.04,0.230425,0.84,15.52,"60,07 %",60.07,61.17,64.90,"64,90 %",70.940209,1,3.190,[✅ Distribuição de rendimentos consistente],"[❌ Preço fora da faixa saudável, ❌ Liquidez in...",1


In [15]:
import pandas as pd
from glob import glob

# =====================================================
# CARTEIRA
# =====================================================
carteira = {
    'BTAL11': {'tipo': 'Agro', 'quantidade': 63, 'preco_medio': 78.50},
    'CACR11': {'tipo': 'Papel', 'quantidade': 68, 'preco_medio': 73.94},
    'KNCR11': {'tipo': 'Papel', 'quantidade': 20, 'preco_medio': 103.94},
    'LIFE11': {'tipo': 'Hospitalar', 'quantidade': 610, 'preco_medio': 8.13},
    'RURA11': {'tipo': 'Tijolo', 'quantidade': 290, 'preco_medio': 8.61},
    'RZAG11': {'tipo': 'Fiagro', 'quantidade': 270, 'preco_medio': 9.49},
    'RZAK11': {'tipo': 'Papel', 'quantidade': 124, 'preco_medio': 82.52},
    'SPXS11': {'tipo': 'Multi', 'quantidade': 590, 'preco_medio': 8.40},
}

# =====================================================
# LOAD BASE
# =====================================================
base_completa = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')

dfs = sorted(glob('df_fiis/df_fiis_*'))
dia_anterior = dfs[-2].split('.')[0].split('_')[-1]
dia_hoje = dfs[-1].split('.')[0].split('_')[-1]

metricas = ['Preço Atual (R$)', 'Score']

# =====================================================
# FUNÇÕES
# =====================================================
def formatar_variacao(anterior, atual, metrica):
    delta = atual - anterior
    pct = (delta / anterior) * 100 if anterior != 0 else 0

    if delta > 0:
        seta, emoji = "↑", "🟢"
    elif delta < 0:
        seta, emoji = "↓", "🔴"
    else:
        seta, emoji = "→", "🟡"

    if metrica == "Score":
        return f"{emoji} {anterior:.0f} → {atual:.0f} ({seta} {delta:+.0f})"
    else:
        return f"{emoji} {anterior:.2f} → {atual:.2f} ({seta} {delta:+.2f} | {pct:+.1f}%)"


def resultado_fii(qtd, preco_medio, preco_ontem, preco_hoje):
    resultado_dia = (preco_hoje - preco_ontem) * qtd
    resultado_total = (preco_hoje - preco_medio) * qtd

    pct_dia = (preco_hoje - preco_ontem) / preco_ontem * 100 if preco_ontem != 0 else 0
    pct_total = (preco_hoje - preco_medio) / preco_medio * 100 if preco_medio != 0 else 0

    return resultado_dia, resultado_total, pct_dia, pct_total

# =====================================================
# COMPARATIVO DE MÉTRICAS
# =====================================================
print(f"\n📊 COMPARATIVO {dia_anterior} → {dia_hoje}\n")

for fii in carteira.keys():
    print(f"🔹 {fii}")
    for metrica in metricas:
        anterior = base_completa[
            (base_completa.data == dia_anterior) & (base_completa.Fundos == fii)
        ][metrica].iloc[0]

        atual = base_completa[
            (base_completa.data == dia_hoje) & (base_completa.Fundos == fii)
        ][metrica].iloc[0]
        movimentacao = formatar_variacao(anterior, atual, metrica)

        if movimentacao.split(' ')[0] != '🟡':
            print(f"   • {metrica}: {movimentacao}")
    print("")

# =====================================================
# RESULTADO DA CARTEIRA
# =====================================================
print("\n💰 RESULTADO DA CARTEIRA\n")

resultado_carteira_dia = 0
resultado_carteira_total = 0
valor_carteira_ontem = 0
custo_total_carteira = 0

for fii, info in carteira.items():
    qtd = info['quantidade']
    preco_medio = info['preco_medio']

    preco_ontem = base_completa[
        (base_completa.data == dia_anterior) & (base_completa.Fundos == fii)
    ]['Preço Atual (R$)'].iloc[0]

    preco_hoje = base_completa[
        (base_completa.data == dia_hoje) & (base_completa.Fundos == fii)
    ]['Preço Atual (R$)'].iloc[0]

    res_dia, res_total, pct_dia, pct_total = resultado_fii(
        qtd, preco_medio, preco_ontem, preco_hoje
    )

    resultado_carteira_dia += res_dia
    resultado_carteira_total += res_total
    valor_carteira_ontem += preco_ontem * qtd
    custo_total_carteira += preco_medio * qtd

    emoji_dia = "🟢" if res_dia > 0 else "🔴" if res_dia < 0 else "🟡"
    emoji_total = "🟢" if res_total > 0 else "🔴" if res_total < 0 else "🟡"

    print(f"🔹 {fii}")
    print(f"   • Resultado do dia: {emoji_dia} R$ {res_dia:+.2f} ({pct_dia:+.2f}%)")
    print(f"   • Resultado total: {emoji_total} R$ {res_total:+.2f} ({pct_total:+.2f}%)\n")

# =====================================================
# RESUMO + VALORIZAÇÃO %
# =====================================================
valorizacao_dia_pct = (resultado_carteira_dia / valor_carteira_ontem) * 100 if valor_carteira_ontem else 0
valorizacao_total_pct = (resultado_carteira_total / custo_total_carteira) * 100 if custo_total_carteira else 0

print("📊 RESUMO DA CARTEIRA")
print(f"• Resultado do dia: R$ {resultado_carteira_dia:+.2f}")
print(f"• Resultado total vs preço médio: R$ {resultado_carteira_total:+.2f}")
print("\n📈 VALORIZAÇÃO DA CARTEIRA")

print(f"• Valorização do dia: {valorizacao_dia_pct:+.2f}%")
print(f"• Valorização total vs preço médio: {valorizacao_total_pct:+.2f}%")



📊 COMPARATIVO 2026-01-08 → 2026-01-09

🔹 BTAL11
   • Preço Atual (R$): 🟢 85.03 → 85.47 (↑ +0.44 | +0.5%)

🔹 CACR11
   • Preço Atual (R$): 🔴 77.96 → 77.95 (↓ -0.01 | -0.0%)

🔹 KNCR11
   • Preço Atual (R$): 🟢 105.15 → 105.75 (↑ +0.60 | +0.6%)

🔹 LIFE11
   • Preço Atual (R$): 🟢 8.25 → 8.32 (↑ +0.07 | +0.8%)

🔹 RURA11
   • Preço Atual (R$): 🔴 8.68 → 8.63 (↓ -0.05 | -0.6%)

🔹 RZAG11
   • Preço Atual (R$): 🔴 8.92 → 8.39 (↓ -0.53 | -5.9%)

🔹 RZAK11
   • Preço Atual (R$): 🔴 84.65 → 84.56 (↓ -0.09 | -0.1%)

🔹 SPXS11
   • Preço Atual (R$): 🔴 9.10 → 8.91 (↓ -0.19 | -2.1%)


💰 RESULTADO DA CARTEIRA

🔹 BTAL11
   • Resultado do dia: 🟢 R$ +27.72 (+0.52%)
   • Resultado total: 🟢 R$ +439.11 (+8.88%)

🔹 CACR11
   • Resultado do dia: 🔴 R$ -0.68 (-0.01%)
   • Resultado total: 🟢 R$ +272.68 (+5.42%)

🔹 KNCR11
   • Resultado do dia: 🟢 R$ +12.00 (+0.57%)
   • Resultado total: 🟢 R$ +36.20 (+1.74%)

🔹 LIFE11
   • Resultado do dia: 🟢 R$ +42.70 (+0.85%)
   • Resultado total: 🟢 R$ +115.90 (+2.34%)

🔹 RURA11
   • 

In [47]:
import pandas as pd
from glob import glob

# =====================================================
# CARTEIRA
# =====================================================
carteira = {
    'BTAL11': {'tipo': 'Agro', 'quantidade': 63, 'preco_medio': 78.50},
    'CACR11': {'tipo': 'Papel', 'quantidade': 68, 'preco_medio': 73.94},
    'KNCR11': {'tipo': 'Papel', 'quantidade': 20, 'preco_medio': 103.94},
    'LIFE11': {'tipo': 'Hospitalar', 'quantidade': 610, 'preco_medio': 8.13},
    'RURA11': {'tipo': 'Tijolo', 'quantidade': 290, 'preco_medio': 8.61},
    'RZAK11': {'tipo': 'Papel', 'quantidade': 124, 'preco_medio': 82.52},
    'SPXS11': {'tipo': 'Multi', 'quantidade': 590, 'preco_medio': 8.40},
}

# =====================================================
# LOAD BASE
# =====================================================
base = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
base = base.sort_values(['Fundos', 'data'])

# =====================================================
# FUNÇÃO DE EMOJI
# =====================================================
def emoji_valor(v):
    if v > 0:
        return "🟢"
    elif v < 0:
        return "🔴"
    else:
        return "🟡"

# =====================================================
# VARIAÇÃO DIÁRIA POR FII
# =====================================================
print("\n📊 VARIAÇÃO DIÁRIA POR FII (ÚLTIMOS 3 DIAS)\n")

resultado_carteira_por_dia = {}

for fii, info in carteira.items():
    qtd = info['quantidade']
    preco_medio = info['preco_medio']

    df = base[base['Fundos'] == fii].copy()
    df['preco_anterior'] = df['Preço Atual (R$)'].shift(1)
    df = df.dropna()

    last_3_days = df['data'].unique()[-3:]
    df = df[df['data'].isin(last_3_days)]

    df['variacao_pct'] = (df['Preço Atual (R$)'] - df['preco_anterior']) / df['preco_anterior'] * 100
    df['resultado_dia'] = (df['Preço Atual (R$)'] - df['preco_anterior']) * qtd

    acumulado_3d = df['resultado_dia'].sum()
    pct_3d = acumulado_3d / (qtd * preco_medio) * 100

    print(f"🔹 {fii}")

    for _, row in df.iterrows():
        data = row['data']
        res_dia = row['resultado_dia']
        pct = row['variacao_pct']

        print(
            f"   {data} → "
            f"{emoji_valor(res_dia)} {pct:+.2f}% | "
            f"R$ {res_dia:+.2f}"
        )

        resultado_carteira_por_dia[data] = (
            resultado_carteira_por_dia.get(data, 0) + res_dia
        )

    print(
        f"   🔸 ACUMULADO 3 DIAS: "
        f"{emoji_valor(acumulado_3d)} R$ {acumulado_3d:+.2f} "
        f"({pct_3d:+.2f}%)\n"
    )

# =====================================================
# RESUMO DA CARTEIRA POR DIA
# =====================================================
print("\n💼 RESULTADO DIÁRIO DA CARTEIRA\n")

acumulado = 0

for data in sorted(resultado_carteira_por_dia.keys()):
    valor = resultado_carteira_por_dia[data]
    acumulado += valor

    print(
        f"{data} → "
        f"{emoji_valor(valor)} R$ {valor:+.2f} | "
        f"Acumulado: R$ {acumulado:+.2f}"
    )

print("\n📈 RESULTADO FINAL DA CARTEIRA (3 DIAS)")
print(f"• Acumulado total: R$ {acumulado:+.2f}")



📊 VARIAÇÃO DIÁRIA POR FII (ÚLTIMOS 3 DIAS)

🔹 BTAL11
   2026-01-06 → 🟡 +0.00% | R$ +0.00
   2026-01-07 → 🟢 +0.06% | R$ +3.15
   2026-01-08 → 🟢 +0.09% | R$ +5.04
   🔸 ACUMULADO 3 DIAS: 🟢 R$ +8.19 (+0.17%)

🔹 CACR11
   2026-01-06 → 🔴 -0.26% | R$ -13.60
   2026-01-07 → 🟢 +0.14% | R$ +7.48
   2026-01-08 → 🟢 +0.04% | R$ +2.04
   🔸 ACUMULADO 3 DIAS: 🔴 R$ -4.08 (-0.08%)

🔹 KNCR11
   2026-01-06 → 🔴 -0.54% | R$ -11.40
   2026-01-07 → 🔴 -0.69% | R$ -14.60
   2026-01-08 → 🔴 -0.05% | R$ -1.00
   🔸 ACUMULADO 3 DIAS: 🔴 R$ -27.00 (-1.30%)

🔹 LIFE11
   2026-01-06 → 🟢 +0.48% | R$ +24.40
   2026-01-07 → 🔴 -1.91% | R$ -97.60
   2026-01-08 → 🟢 +0.24% | R$ +12.20
   🔸 ACUMULADO 3 DIAS: 🔴 R$ -61.00 (-1.23%)

🔹 RURA11
   2026-01-06 → 🟢 +1.51% | R$ +37.70
   2026-01-07 → 🔴 -1.03% | R$ -26.10
   2026-01-08 → 🟢 +0.23% | R$ +5.80
   🔸 ACUMULADO 3 DIAS: 🟢 R$ +17.40 (+0.70%)

🔹 RZAK11
   2026-01-06 → 🟢 +0.64% | R$ +65.72
   2026-01-07 → 🟢 +1.36% | R$ +141.36
   2026-01-08 → 🔴 -0.15% | R$ -16.12
   🔸 ACUMULADO 3 D